In [1]:
import jieba
import logging
import numpy as np
from gensim.models import word2vec
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors

Using TensorFlow backend.
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

In [2]:
def jieba_tokenizer(content):
    # load stopwords set
    stopword_set = set()
    with open('jieba_dict/stopwords.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip(' \n'))

    if str(content)=='nan':
        return ''
    words = jieba.cut(content.replace(' ',''))
    return [word for word in words if word not in stopword_set]

In [4]:
def word2token(word,w2v_model):

    # Retrieve the weights from the model. This is used for initializing the weights
    # in a Keras Embedding layer later
    w2v_weights = w2v_model.wv.vectors
    vocab_size, embedding_size = w2v_weights.shape

    try:
        return w2v_model.wv.vocab[word].index
    # If word is not in index return 0. I realize this means that this
    # is the same as the word of index 0 (i.e. most frequent word), but 0s
    # will be padded later anyway by the embedding layer (which also
    # seems dirty but I couldn't find a better solution right now)
    except KeyError:
        return 0
    
def token2word(token):
    return w2v_model.wv.index2word[token]

In [7]:
def suicide_predict(text):
    model = load_model('lstm_model.h5')

    # jieba custom setting.
    jieba.set_dictionary('jieba_dict/dict.txt.big')
    text_token = jieba_tokenizer(text)
    
    
    w2v_model = KeyedVectors.load("w2v_model.wv", mmap='r')   
    
    for i in range(len(text_token)):
        text_token[i] = word2token(text_token[i],w2v_model)

    text_token = pad_sequences([text_token], maxlen=250, padding='post', value=0)

    score = model.predict(text_token)
    if score[0][0] > score[0][1]:
        results = 0
    else:
        results = 1

    return results

In [10]:
suicide_predict("也許雨季來臨前就開始了吧什麼都不想做只覺得好累好累考完駕照雖然開心但更覺得鬆了一口氣終於不用在約定時間跟人見面要讀的paper越積越多進度仍是零雖然可以隨便做做但還是想試著做好想試著做好卻一直無法專心讀著下禮拜要上臺介紹的散文集羨慕那些可以有辦法寫的人而我寫的東西永遠詞不達義--")

Building prefix dict from /home/ubuntu-gpu/Project_Depression/jieba_dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u156e2db6fd044c50a8c659fc2a66f9b6.cache
Loading model cost 1.295 seconds.
Prefix dict has been built succesfully.
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


0